# Subset MODIS Tiles from NASA's Earthdata Cloud
In this tutorial, we will subset a MODIS Land Product (MOD13Q1) available in the Earthdata Cloud. You will need to be in AWS us-west region.

To access a Juypter Notebook on [AWS EC2](https://aws.amazon.com/ec2/) instance:
1. On the AWS EC2 instance, start Juypter Notebook on port 8888 with the 'no-browser' parameter: `jupyter notebook --no-browser --port=8888`
1. On your local machine, forward port 8000 to the remote port 8888 : `ssh -i my-key-pair.pem -L 8000:localhost:8888 my-instance-user-name@my-instance-IPv6-address`
1. Now, the Jupyter Notebook can be accessed at `http://localhost:8000` in your local browser.

In [1]:
import requests # request==2.28.1 
import math 
import datetime as dt
import numpy as np
from requests.adapters import HTTPAdapter, Retry
import s3fs # s3fs==0.4.2 
import rioxarray as rxr # rioxarray==0.13.3 
from posixpath import splitext
from os import path, remove
import pandas as pd # pandas==1.4.4

### Product details

In [2]:
# product 
product = "MOD13Q1"

# file format
f_format = 'hdf'

# product doi
doi = "10.5067/MODIS/MOD13Q1.061"

# product resolution in meters
grid = 250

# root data directory
data_path = "MODIS_Grid_16DAY_250m_500m_VI"

### Subset parameters

In [3]:
# latitude/logitude in degrees decimals

lat = 39.96852 # bottom /left span
lon = -91.24829

# subset size in km
kmLeftRight = 1
kmAboveBelow = 1

# start & end dates
start_date = dt.datetime(2016, 12, 15) 
end_date = dt.datetime(2017, 1, 15)

# variables
variables = ['250m 16 days NDVI',
 '250m 16 days relative azimuth angle',
 '250m 16 days composite day of the year',
 '250m 16 days pixel reliability',
 '250m 16 days EVI',
 '250m 16 days VI Quality',
 '250m 16 days red reflectance',
 '250m 16 days NIR reflectance',
 '250m 16 days blue reflectance',
 '250m 16 days MIR reflectance',
 '250m 16 days view zenith angle',
 '250m 16 days sun zenith angle']

### Tilemapper function

In [4]:
# returns vert tile, horiz tile, line, samp
def tilemapper(laty, lonx, res):
    dim = math.ceil(4800*250/res)
    y = ((90 - laty)/180 * 18 * dim - 0.5) 
    x = (lonx/360*36*dim * math.cos(laty * np.pi / 180) + 18*dim - .5) % (36*dim)
    t_h = math.floor(x/dim)
    t_v = math.floor(y/dim)
    t_s = math.ceil(x % dim)
    t_l = math.ceil(y % dim)
    if ((t_l-dim)==0):
        t_l=0
        t_v=t_v+1
    if ((t_s-dim)==0):
        t_s=0
        t_h=t_h+1
    return (dim, t_v, t_h, t_l, t_s)

(modis_tile_size, v, h, l, s) = tilemapper(lat, lon, grid)

# print
print(f"{modis_tile_size=}, {v=}, {h=}, {l=}, {s=}")

modis_tile_size=4800, v=5, h=11, l=15, s=32


### Calculate box size

In [5]:
# number of pixels
pix_left_right = math.ceil(kmLeftRight * 1000/grid)
pix_above_below = math.ceil(kmAboveBelow * 1000/grid)

# if s, l are negative, there is a jump
s = s - pix_left_right 
l = l - pix_above_below

h_span = h
v_span = v
line_span = l
samp_span = s

# tile jump
jump = 0
if (s < 0):
    jump = 1
    h_span = h-1
    samp_span = modis_tile_size+s # NW pixel
    
if (l < 0):
    jump = 1
    v_span = v-1
    line_span = modis_tile_size+l # NW pixel

box_wanted_samp = pix_left_right+pix_left_right + 1
box_wanted_line = pix_above_below+pix_above_below + 1 

# tile span
span = 0
calc_samp = box_wanted_samp + samp_span
calc_line = box_wanted_line + line_span
if (calc_samp > modis_tile_size):
    span = 1
if (calc_line > modis_tile_size):
    span = 1
mts_plus1 = modis_tile_size + 1

# print
print(f"{line_span=}, {box_wanted_samp=}, {box_wanted_line=}, {jump=}, {span=}, {calc_samp=}, {calc_line=}")

line_span=11, box_wanted_samp=9, box_wanted_line=9, jump=0, span=0, calc_samp=37, calc_line=20


### Compute line, sample spans for tiles

In [6]:
def hv_str(h_int, v_int):
    return f"h{str(h_int).zfill(2)}v{str(v_int).zfill(2)}"

def get_tile_info(t_pos, t_h, t_v, t_l, t_s, t_box_l, t_box_s):    
    t_tile = hv_str(t_h, t_v)
    return {'order' : t_pos,
            'tile': t_tile, 
            'l': t_l, 
            's': t_s,
            'box_line': t_box_l,
            'box_samp': t_box_s}
    
def get_box_line_samp(cl, cs, mts):
    hv = []
    # bottom and left span
    if ((cs > mts) and (cl > mts)): 
        # nw
        hv.append(get_tile_info(0, h_span, v_span, line_span, samp_span, 
                          modis_tile_size - line_span, modis_tile_size - samp_span))
        # ne
        hv.append(get_tile_info(1, h_span + 1, v_span, line_span, 0, 
                          modis_tile_size - line_span, calc_samp - modis_tile_size))    
        # sw
        hv.append(get_tile_info(2, h_span, v_span + 1, 0, samp_span, 
                          calc_line - modis_tile_size, modis_tile_size - samp_span))
        # se
        hv.append(get_tile_info(3, h_span+1, v_span + 1, line_span, 0, 
                          calc_line - modis_tile_size, calc_samp - modis_tile_size))
        
    # left span   
    elif ((cs > mts) and (cl <= mts)):
        # nw
        hv.append(get_tile_info(0, h_span, v_span, line_span, samp_span, 
                          box_wanted_line, modis_tile_size-samp_span))
        # ne
        hv.append(get_tile_info(1, h_span + 1, v_span, line_span, 0, 
                          box_wanted_line, calc_samp - modis_tile_size))    
    
    # bottom span
    elif ((cs <= mts) and (cl > mts)):
        # nw
        hv.append(get_tile_info(0, h_span, v_span, line_span, samp_span, 
                          modis_tile_size - line_span, box_wanted_samp))
        # sw
        hv.append(get_tile_info(1, h_span, v_span + 1, 0, samp_span, 
                          calc_line - modis_tile_size, box_wanted_samp))
    else:
        # no jump
        hv.append(
            get_tile_info(0, h, v, l, s, box_wanted_line, box_wanted_samp)
        )
    return hv

hv = get_box_line_samp (calc_line, calc_samp, modis_tile_size)

hv_df = pd.DataFrame(hv)
hv_df

,order,tile,l,s,box_line,box_samp
0,0,h11v05,11,28,9,9


### Define search granules function

In [7]:
def search_granules(tile_hv, c_id, time_str):
    page_num = 1
    g_arr = []

    while True:
         # defining parameters
        cmr_param = {
            "collection_concept_id": c_id, 
            "temporal": time_str,
            "page_size": 2000,
            "page_num": page_num,
            "options[producerGranuleId][pattern]": "true",
            "producerGranuleId": f"*.{tile_hv}.*"        
        }
        granulesearch = cmrurl + 'granules.json'
        response = requests.get(granulesearch, params=cmr_param)
        granules = response.json()['feed']['entry']
        if granules:
            for g in granules:
                # Get URL of HDF5 files
                for links in g['links']:
                    if links['href'].startswith('s3://') and links['href'].endswith(f".{f_format}"):
                        s3_l = path.basename(links['href']).split('.')
                        g_arr.append([links['href'], path.basename(links['href']), s3_l[1], s3_l[2], int(s3_l[4])])

            page_num += 1
        else: 
            break

    # printing granules
    return g_arr

### Search granules

In [8]:
# cmr api url
cmrurl='https://cmr.earthdata.nasa.gov/search/'
# doi search to get concept_id
doisearch = f'{cmrurl}collections.json?doi={doi}'
concept_id = requests.get(doisearch).json()['feed']['entry'][0]['id']

# CMR datetime format
dt_format = '%Y-%m-%dT%H:%M:%SZ'
# CMR time bounds
temporal = start_date.strftime(dt_format) + ',' + end_date.strftime(dt_format)

granule_arr=[]
for hv_dict in hv:
    granule_arr.extend(search_granules(hv_dict['tile'], concept_id, temporal))

# create pandas dataframe
granule_df = pd.DataFrame(granule_arr, columns=['s3_link', 'granule', 'modis_date', 'hv', 'proc_date'])

# removing any granules with older proc dates
granule_df = granule_df.loc[granule_df.groupby(['modis_date', 'hv'])['proc_date'].idxmax()].reset_index()
granule_df

,index,s3_link,granule,modis_date,hv,proc_date
0,0,s3://lp-prod-protected/MOD13Q1.061/MOD13Q1.A20...,MOD13Q1.A2016337.h11v05.061.2021363040441.hdf,A2016337,h11v05,2021363040441
1,1,s3://lp-prod-protected/MOD13Q1.061/MOD13Q1.A20...,MOD13Q1.A2016353.h11v05.061.2021363152448.hdf,A2016353,h11v05,2021363152448
2,2,s3://lp-prod-protected/MOD13Q1.061/MOD13Q1.A20...,MOD13Q1.A2017001.h11v05.061.2021257162728.hdf,A2017001,h11v05,2021257162728


### Get S3 credentials
We will now get S3 temporary credentials and define the S3FS S3FileSystem object. Make sure that `.netrc` file is defined as described here: https://wiki.earthdata.nasa.gov/display/EL/How+To+Access+Data+With+cURL+And+Wget.

In [9]:
# creating a backoff
session = requests.Session()
retries = Retry(total=3, backoff_factor=0.1, status_forcelist=[ 500, 502, 503, 504 ])
session.mount('https://', HTTPAdapter(max_retries=retries))

# lp daac s3
lp_s3 = f"https://data.lpdaac.earthdatacloud.nasa.gov/s3credentials"

# get s3 credentials
s3credentials = session.get(lp_s3).json()

# defining S3FS object
fs_s3 =  s3fs.S3FileSystem(anon=False, 
                      key=s3credentials['accessKeyId'], 
                      secret=s3credentials['secretAccessKey'], 
                      token=s3credentials['sessionToken'])

### Get Subsets

In [10]:
data = []
for date_uniq in granule_df.modis_date.unique():
    temp_df = granule_df[granule_df.modis_date==date_uniq]
       
    # loop over tiles and download
    for i in hv_df.order:
        tile_name = hv_df[hv_df.order==i].tile.values[0]
        s3_link = temp_df[temp_df.hv==tile_name].s3_link.values[0]   
        g_temp = temp_df[temp_df.hv==tile_name].granule.values[0]  
        fs_s3.download(s3_link, g_temp)
    
    # loop over variables
    for v in variables:
        temp_data = []
        
        for i in hv_df.order:   
            tile_name = hv_df[hv_df.order==i].tile.values[0]
            g_temp = temp_df[temp_df.hv==tile_name].granule.values[0]
            
            # defining subset indices
            l1 = hv_df[hv_df.order==i].l.values[0]
            l2 = l1 + hv_df[hv_df.order==i].box_line.values[0]
            s1 = hv_df[hv_df.order==i].s.values[0]
            s2 = s1 + hv_df[hv_df.order==i].box_samp.values[0]
            if path.isfile(g_temp):
                with rxr.open_rasterio(g_temp) as modis:
                    temp_data.extend(modis[v][:, l1:l2, s1:s2].values.flatten().tolist()) 
        
        # removing spaces from band names
        band = v.replace(" ", "_" )
        data.append([g_temp, band, temp_data])
    
    # delete temp files
    for g_temp in temp_df.granule:
        if path.isfile(g_temp):
            remove(g_temp)
            
headers = ["granule", "band", "data"]

# print the output
pd.DataFrame(data, columns=headers).to_json()

'{"granule":{"0":"MOD13Q1.A2016337.h11v05.061.2021363040441.hdf","1":"MOD13Q1.A2016337.h11v05.061.2021363040441.hdf","2":"MOD13Q1.A2016337.h11v05.061.2021363040441.hdf","3":"MOD13Q1.A2016337.h11v05.061.2021363040441.hdf","4":"MOD13Q1.A2016337.h11v05.061.2021363040441.hdf","5":"MOD13Q1.A2016337.h11v05.061.2021363040441.hdf","6":"MOD13Q1.A2016337.h11v05.061.2021363040441.hdf","7":"MOD13Q1.A2016337.h11v05.061.2021363040441.hdf","8":"MOD13Q1.A2016337.h11v05.061.2021363040441.hdf","9":"MOD13Q1.A2016337.h11v05.061.2021363040441.hdf","10":"MOD13Q1.A2016337.h11v05.061.2021363040441.hdf","11":"MOD13Q1.A2016337.h11v05.061.2021363040441.hdf","12":"MOD13Q1.A2016353.h11v05.061.2021363152448.hdf","13":"MOD13Q1.A2016353.h11v05.061.2021363152448.hdf","14":"MOD13Q1.A2016353.h11v05.061.2021363152448.hdf","15":"MOD13Q1.A2016353.h11v05.061.2021363152448.hdf","16":"MOD13Q1.A2016353.h11v05.061.2021363152448.hdf","17":"MOD13Q1.A2016353.h11v05.061.2021363152448.hdf","18":"MOD13Q1.A2016353.h11v05.061.202136315